In [1]:
import numpy as np

class CellGrid:
    def __init__(self, coords, neighbors, genes, gene_names):
        """
        N: Number of cells on grid 
        G: Number of genes
        coords: np.array [N,2] of x,y positions
        neighbors: np.array [N] number of neighbors per cell
        genes: np.array [N,G] boolean matrix for gene presence
        gene_names: list of gene names corresponding to columns of genes
        """
        self.coords = coords
        self.neighbors = neighbors
        self.genes = genes
        self.gene_names = gene_names
        self.gene_idx = {name: i for i, name in enumerate(gene_names)}

    def get_coords(self, cell_indices=None):
        """Return coordinates of all cells or selected cells."""
        if cell_indices is None:
            return self.coords
        return self.coords[cell_indices]

    def get_genes(self, cell_indices=None):
        """Returns gene matrix for all cells or selected cells"""
        if cell_indices is None:
            return self.genes
        return self.genes[cell_indices]

    def get_full_matrix(self):
        """
        Returns a combined matrix of shape (N, 2 + G)
        Columns: [x, y, gene1, gene2, ..., geneG]
        """
        return np.hstack([self.coords, self.genes])


In [2]:
# Créons des données fictives pour 100 cellules et 20 gènes
N, G = 100, 20
coords = np.random.rand(N, 2)  # x,y entre 0 et 1
neighbors = np.random.randint(1, 5, N)
genes = np.random.randint(0, 2, (N, G))  # 0 ou 1
gene_names = [f"gene_{i}" for i in range(G)]

grid = CellGrid(coords, neighbors, genes, gene_names)

full_matrix = grid.get_full_matrix()
print(full_matrix.shape)  # (100, 22)
print(full_matrix[:5])    # Affiche les 5 premières lignes


(100, 22)
[[0.74450549 0.16091326 1.         0.         1.         0.
  1.         0.         1.         1.         1.         0.
  1.         0.         1.         0.         0.         0.
  1.         1.         1.         1.        ]
 [0.22955516 0.27783892 1.         1.         0.         1.
  0.         0.         1.         0.         1.         1.
  1.         0.         0.         0.         1.         0.
  0.         1.         0.         1.        ]
 [0.07324086 0.36305469 1.         1.         1.         0.
  1.         1.         0.         1.         1.         0.
  0.         0.         1.         1.         1.         1.
  1.         1.         1.         0.        ]
 [0.661858   0.05965584 1.         1.         1.         1.
  0.         1.         0.         1.         0.         0.
  0.         0.         1.         0.         0.         0.
  0.         0.         0.         1.        ]
 [0.59389188 0.02339914 1.         0.         1.         0.
  0.         1.       

In [3]:
class CellGrid:
    def __init__(self, coords, neighbors, genes, gene_names):
        self.coords = coords
        self.neighbors = neighbors
        self.genes = genes
        self.gene_names = gene_names
        self.gene_idx = {name: i for i, name in enumerate(gene_names)}
        self.N = coords.shape[0]
        self.G = genes.shape[1]

        # Initial cell status: 1 = alive, 0 = dead
        self.alive = np.ones(self.N, dtype=int)  

    def get_full_matrix(self):
        """Combine coords + genes + status"""
        return np.hstack([self.coords, self.alive.reshape(-1,1), self.genes])

    def update_cells(self):
        """Exemple de boucle pour mettre à jour le statut et les gènes"""
        # Boucle sur les cellules
        for i in range(self.N):
            # 1. Appliquer la règle des voisins pour décider si la cellule reste vivante
            self.alive[i] = self.alive_rule(i)

            # 2. Mettre à jour les gènes selon le contenu ou autres règles
            self.genes[i] = self.content_rule(i)

    def alive_rule(self, idx):
        """Déterminer si la cellule idx reste vivante (exemple basique)"""
        # ici tu peux utiliser self.neighbors et self.alive
        num_alive_neighbors = np.sum(self.alive[self.neighbors[idx]])  # si neighbors[idx] est une liste d'indices
        if self.alive[idx] == 1 and num_alive_neighbors < 2:
            return 0  # mort par sous-population
        elif self.alive[idx] == 1 and num_alive_neighbors > 3:
            return 0  # mort par surpopulation
        elif self.alive[idx] == 0 and num_alive_neighbors == 3:
            return 1  # naissance
        return self.alive[idx]

    def content_rule(self, idx):
        """Mettre à jour les gènes de la cellule idx selon des règles"""
        # exemple: activer le gène 0 si la cellule est vivante
        new_genes = self.genes[idx].copy()
        if self.alive[idx] == 1:
            new_genes[0] = 1
        return new_genes


In [4]:
N, G = 100, 20

# Vérifions les données
coords = np.random.rand(N,2)
neighbors = [list(np.random.choice(N, size=np.random.randint(1,5), replace=False)) for _ in range(N)]
genes = np.random.randint(0,2,(N,G))
gene_names = [f"gene_{i}" for i in range(G)]

# Créons le grid
grid = CellGrid(coords, neighbors, genes, gene_names)

# Vérification dimensions
print("coords shape:", grid.coords.shape)
print("neighbors length:", len(grid.neighbors))
print("genes shape:", grid.genes.shape)
print("gene_names length:", len(grid.gene_names))
print("alive shape:", grid.alive.shape)

# Vérifions la matrice complète
full_matrix = grid.get_full_matrix()
print("full_matrix shape:", full_matrix.shape)
print("Exemple full_matrix (5 premières lignes):\n", full_matrix[:5])

# Test d'update
grid.update_cells()
print("Après update, alive[:5]:", grid.alive[:5])
print("Après update, genes[:5]:\n", grid.genes[:5])


coords shape: (100, 2)
neighbors length: 100
genes shape: (100, 20)
gene_names length: 20
alive shape: (100,)
full_matrix shape: (100, 23)
Exemple full_matrix (5 premières lignes):
 [[0.26362064 0.61949789 1.         0.         1.         0.
  0.         0.         0.         0.         1.         1.
  0.         1.         0.         0.         0.         1.
  0.         0.         0.         0.         1.        ]
 [0.86980845 0.824112   1.         0.         0.         0.
  0.         0.         1.         0.         1.         0.
  1.         0.         0.         0.         0.         1.
  0.         0.         0.         1.         1.        ]
 [0.62772309 0.75989377 1.         0.         0.         1.
  1.         0.         1.         0.         1.         1.
  1.         0.         1.         0.         0.         0.
  0.         0.         1.         1.         1.        ]
 [0.55247472 0.54961341 1.         0.         1.         0.
  1.         1.         1.         1.       

In [5]:
# Boucle d'évolution exemple
print("=== Avant évolution ===")
print("Alive (5 premières cellules):", grid.alive[:5])
print("Genes (5 premières cellules, gènes 0 à 4):\n", grid.genes[:5,:5])

# Appliquer une évolution
grid.update_cells()

print("\n=== Après 1 itération d'évolution ===")
print("Alive (5 premières cellules):", grid.alive[:5])
print("Genes (5 premières cellules, gènes 0 à 4):\n", grid.genes[:5,:5])

# Afficher full_matrix pour vérifier
full_matrix_after = grid.get_full_matrix()
print("\nFull matrix shape:", full_matrix_after.shape)
print("Exemple full_matrix (5 premières lignes):\n", full_matrix_after[:5])


=== Avant évolution ===
Alive (5 premières cellules): [0 0 1 1 0]
Genes (5 premières cellules, gènes 0 à 4):
 [[0 1 0 0 0]
 [0 0 0 0 0]
 [1 0 1 1 0]
 [1 1 0 1 1]
 [1 1 0 1 1]]

=== Après 1 itération d'évolution ===
Alive (5 premières cellules): [0 0 0 0 0]
Genes (5 premières cellules, gènes 0 à 4):
 [[0 1 0 0 0]
 [0 0 0 0 0]
 [1 0 1 1 0]
 [1 1 0 1 1]
 [1 1 0 1 1]]

Full matrix shape: (100, 23)
Exemple full_matrix (5 premières lignes):
 [[0.26362064 0.61949789 0.         0.         1.         0.
  0.         0.         0.         0.         1.         1.
  0.         1.         0.         0.         0.         1.
  0.         0.         0.         0.         1.        ]
 [0.86980845 0.824112   0.         0.         0.         0.
  0.         0.         1.         0.         1.         0.
  1.         0.         0.         0.         0.         1.
  0.         0.         0.         1.         1.        ]
 [0.62772309 0.75989377 0.         1.         0.         1.
  1.         0.         

In [10]:
import numpy as np

N = 100   # nombre de cellules
P = 20    # positions / slots par cellule
G = 3     # x, y, type

# Grille vide initialisée à 0
grid_3d = np.zeros((N, P, G), dtype=int)

print("Grille 3D shape:", grid_3d.shape)
print("Exemple (cellule 0, premières positions):\n", grid_3d[0, :5, :])


Grille 3D shape: (100, 20, 3)
Exemple (cellule 0, premières positions):
 [[0 0 0]
 [0 0 0]
 [0 0 0]
 [0 0 0]
 [0 0 0]]


In [27]:
import numpy as np

def init_position_gene_grid(X, Y, gene_list):
    """
    Initialise une grille 3D (X, Y, G) avec 0/1 selon les gènes actifs à chaque position.
    
    Paramètres :
    -----------
    X : int
        Taille en x (ex : 100)
    Y : int
        Taille en y (ex : 20)
    gene_list : list of tuples
        Chaque tuple = (x, y, g_idx)
        - x, y : coordonnées de la position
        - g_idx : numéro du gène à activer
        
    Retour :
    --------
    grid : np.array
        Grille 3D (X, Y, G) avec 0/1
    """
    if len(gene_list) == 0:
        raise ValueError("La liste de gènes ne peut pas être vide")
    
    # Déterminer le nombre de gènes dynamiquement
    G = max([g[2] for g in gene_list]) + 1
    grid = np.zeros((X, Y, G), dtype=int)
    
    # Remplir la grille avec les gènes actifs
    for x, y, g_idx in gene_list:
        if 0 <= x < X and 0 <= y < Y:
            grid[x, y, g_idx] = 1
        else:
            print(f"Avertissement : position ({x},{y}) hors grille, ignorée")
    
    return grid


In [28]:
X, Y = 100, 20

# Liste de gènes actifs : (x, y, g_idx)
genes_to_activate = [
    (42, 5, 3),
    (10, 15, 0),
    (55, 12, 2),
    (42, 5, 0)  # plusieurs gènes pour la même position
]

grid = init_position_gene_grid(X, Y, genes_to_activate)

print("Grille shape :", grid.shape)  # (100, 20, 4) ici G=4 car max(g_idx)=3
print("Gènes à la position (42,5) :", grid[42,5,:])  
# Sortie exemple : [1 0 0 1]


Grille shape : (100, 20, 4)
Gènes à la position (42,5) : [1 0 0 1]


In [29]:
import numpy as np

# Dimensions
X, Y = 100, 20  # x = cellule, y = position
G = 5           # nombre de gènes

# 1️⃣ Grille vide pour initial cell status
grid = np.zeros((X, Y, G), dtype=int)

# 2️⃣ Définir des gènes initiaux pour quelques positions
initial_genes = [
    (0, 0, 0),
    (0, 1, 1),
    (42, 5, 3),
    (42, 5, 0),
    (55, 10, 2)
]

for x, y, g_idx in initial_genes:
    grid[x, y, g_idx] = 1

# 3️⃣ Matrice des gènes (content)
gene_names = ["gène0","gène1","gène2","gène3","gène4"]
gene_matrix = np.zeros((X, Y, G), dtype=int)

# Copier le contenu de la grille dans la matrice de gènes
gene_matrix = grid.copy()


In [30]:
print("Gènes actifs à (42,5) :", grid[42,5,:])
# Exemple sortie : [1 0 0 1 0] → gènes 0 et 3 actifs


Gènes actifs à (42,5) : [1 0 0 1 0]


In [31]:
positions_g3 = np.where(grid[:,:,3]==1)
print("Positions gène 3 actif :", positions_g3)


Positions gène 3 actif : (array([42]), array([5]))


In [34]:
import numpy as np

def init_cell_status(X, Y, G, initial_genes, gene_names=None):
    """
    Initialise une grille spatiale 3D (x, y, gènes) représentant
    l'état génétique des cellules.

    Paramètres :
    -----------
    X : int
        Taille de la grille selon l'axe x (coordonnée spatiale)
    Y : int
        Taille de la grille selon l'axe y (coordonnée spatiale)
    G : int
        Nombre de gènes
    initial_genes : list of tuples
        Chaque tuple = (x, y, g_idx)
        Indique que le gène g_idx est actif (1)
        à la position spatiale (x, y)
    gene_names : list of str, optionnel
        Noms des gènes, longueur G

    Retour :
    --------
    grid : np.ndarray
        Grille 3D de forme (X, Y, G)
        grid[x, y, g] = 1 si le gène g est actif à la position (x, y)
    gene_matrix : np.ndarray
        Copie de grid (contenu génétique initial)
    """
    
    # Vérification
    if gene_names is not None and len(gene_names) != G:
        raise ValueError("gene_names doit avoir une longueur égale à G")
    
    # Initialiser la grille et la matrice de gènes
    grid = np.zeros((X, Y, G), dtype=int)
    gene_matrix = np.zeros((X, Y, G), dtype=int)
    
    # Activer les gènes selon la liste initiale
    for x, y, g_idx in initial_genes:
        if 0 <= x < X and 0 <= y < Y and 0 <= g_idx < G:
            grid[x, y, g_idx] = 1
            gene_matrix[x, y, g_idx] = 1
        else:
            print(f"Avertissement : position ({x},{y},{g_idx}) hors limites, ignorée")
    
    return grid, gene_matrix


In [35]:
X, Y, G = 100, 20, 5

# Gènes actifs initiaux : (x, y, g_idx)
initial_genes = [
    (0, 0, 0),
    (0, 1, 1),
    (42, 5, 3),
    (42, 5, 0),
    (55, 10, 2)
]

# Noms optionnels
gene_names = ["g0","g1","g2","g3","g4"]

grid, gene_matrix = init_cell_status(X, Y, G, initial_genes, gene_names)

print("Grille shape :", grid.shape)       # (100, 20, 5)
print("Gènes actifs à (42,5) :", grid[42,5,:])  # [1 0 0 1 0]


Grille shape : (100, 20, 5)
Gènes actifs à (42,5) : [1 0 0 1 0]


In [37]:
import numpy as np

def init_system(X, Y, G, initial_cells, initial_genes):
    """
    X, Y : dimensions spatiales
    G    : nombre de gènes
    initial_cells : list of (x,y) cellules vivantes
    initial_genes : list of (x,y,g) gènes présents
    """

    cell_status = np.zeros((X, Y), dtype=int)
    gene_content = np.zeros((X, Y, G), dtype=int)

    # Activer cellules
    for x, y in initial_cells:
        cell_status[x, y] = 1

    # Activer gènes uniquement si la cellule existe
    for x, y, g in initial_genes:
        if cell_status[x, y] == 1:
            gene_content[x, y, g] = 1

    return cell_status, gene_content


CODE FINAL

In [39]:
import numpy as np

class CellGrid:
    def __init__(self, X, Y, G, initial_cells=None, initial_genes=None, gene_names=None):
        """
        Grille spatiale de cellules avec contenu génétique.

        Paramètres
        ----------
        X : int
            Taille spatiale en x
        Y : int
            Taille spatiale en y
        G : int
            Nombre de gènes
        initial_cells : list of tuples (x,y), optionnel
            Positions initiales des cellules vivantes
        initial_genes : list of tuples (x,y,g), optionnel
            Gènes présents initialement dans les cellules
        gene_names : list of str, optionnel
            Noms des gènes
        """

        self.X = X
        self.Y = Y
        self.G = G

        # 1️⃣ Cellules vivantes / mortes
        self.cell_status = np.zeros((X, Y), dtype=int)

        # 2️⃣ Contenu génétique
        self.gene_content = np.zeros((X, Y, G), dtype=int)

        # Noms des gènes
        if gene_names is None:
            self.gene_names = [f"gene_{i}" for i in range(G)]
        else:
            if len(gene_names) != G:
                raise ValueError("gene_names doit être de longueur G")
            self.gene_names = gene_names

        # Initialisation des cellules vivantes
        if initial_cells is not None:
            for x, y in initial_cells:
                if self._in_bounds(x, y):
                    self.cell_status[x, y] = 1

        # Initialisation des gènes
        if initial_genes is not None:
            for x, y, g in initial_genes:
                if self._in_bounds(x, y) and 0 <= g < G:
                    if self.cell_status[x, y] == 1:
                        self.gene_content[x, y, g] = 1

    # -------------------------
    # MÉTHODES UTILES
    # -------------------------

    def _in_bounds(self, x, y):
        return 0 <= x < self.X and 0 <= y < self.Y

    def is_alive(self, x, y):
        return self.cell_status[x, y] == 1

    def get_genes(self, x, y):
        """Retourne le vecteur de gènes d'une cellule"""
        return self.gene_content[x, y, :]

    def kill_cell(self, x, y):
        """Tue une cellule et supprime ses gènes"""
        self.cell_status[x, y] = 0
        self.gene_content[x, y, :] = 0

    def add_gene(self, x, y, g):
        """Ajoute un gène à une cellule vivante"""
        if self.is_alive(x, y):
            self.gene_content[x, y, g] = 1

    def remove_gene(self, x, y, g):
        """Supprime un gène"""
        self.gene_content[x, y, g] = 0

    def alive_cells(self):
        """Retourne les coordonnées des cellules vivantes"""
        return np.argwhere(self.cell_status == 1)

    def gene_map(self, g):
        """Carte spatiale d'un gène"""
        return self.gene_content[:, :, g]


In [40]:
X, Y, G = 100, 20, 5

initial_cells = [
    (10, 5),
    (42, 5),
    (50, 10)
]

initial_genes = [
    (10, 5, 0),
    (10, 5, 2),
    (42, 5, 3),
    (50, 10, 1)
]

grid = CellGrid(X, Y, G, initial_cells, initial_genes)

print("Cellule (10,5) vivante ?", grid.is_alive(10,5))
print("Gènes de (10,5) :", grid.get_genes(10,5))
print("Carte du gène 3 :", grid.gene_map(3))


Cellule (10,5) vivante ? True
Gènes de (10,5) : [1 0 1 0 0]
Carte du gène 3 : [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
